In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *

In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_100deg_uniform.npy')
pm_data_noerr = np.load('npy/pm_LAE_100deg_uniform.npy')
f_name = 'Source_cats/Source_cat_LAE_100deg_uniform.npy'
mock = np.load(f_name, allow_pickle=True).item()

errors = np.load('npy/median_errors.npy')

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
N_nb = 5
ew0min = 10

z_idx = np.zeros(mock['redshift_Lya_Arr'].shape).astype(bool)

for nb_c in range(6, 20):
    z_nb = w_central[nb_c] / 1215.67 - 1

    pm_err = (np.ones(pm_data.shape).T * errors).T
    cont_est, cont_err = stack_estimation(
        pm_data, pm_err, nb_c, N_nb, w_central
        )

    diff_err = (errors[nb_c]**2 + cont_err**2)**0.5
    fwhm = nb_fwhm(tcurves, nb_c, True)

    line = (
    (pm_data[nb_c] - cont_est > 3*diff_err)
    & (pm_data[nb_c] - cont_est > ew0min * (1 + z_nb) * cont_est / fwhm)
    )

    z_idx[line] = True

fig, ax = plt.subplots(figsize=(12, 9))
ax.hist(
    mock['redshift_Lya_Arr'][z_idx], bins=np.linspace(2.5, 3.2, 40),
    label='Selected'
)
ax.hist(
    mock['redshift_Lya_Arr'], bins=np.linspace(2.5, 3.2, 40), alpha=0.5, zorder=-1,
    label='All'
    )
for nb_c in range(6, 15):
    ax.axvline(x=w_central[nb_c] / 1215.67 - 1, color='red')
ax.legend(fontsize=15)
ax.set_xlabel('z', fontsize=15)
plt.show()
print('TOTAL SELECTED: {0}/{1} ({2:0.1f}%)'.format(
    len(np.where(z_idx)[0]), len(z_idx), len(np.where(z_idx)[0])/len(z_idx) * 100
    ))

In [ ]:
for nb_c in range(6, 20):
    z_nb = w_central[nb_c] / 1215.67 - 1

    pm_err = (np.ones(pm_data.shape).T * errors).T
    cont_est, cont_err = stack_estimation(
        pm_data, pm_err, nb_c, N_nb, w_central
        )

    diff_err = (errors[nb_c]**2 + cont_err**2)**0.5
    fwhm = nb_fwhm(tcurves, nb_c, True)

    line = (
    (pm_data[nb_c] - cont_est > 3*diff_err)
    & (pm_data[nb_c] - cont_est > ew0min * (1 + z_nb) * cont_est / fwhm)
    )